## Heuristic 4 - Preliminary implementation

### Description
If there are multiple (say 12) deposit transactions coming from a deposit address and later there are 12 withdraw transactions to the same withdraw address, then we can link all these deposit transactions to the withdraw transactions.

In particular, given a withdrawal transaction, an anonimity score is assigned to it:

1) The number of previous withdrawal transactions with the same address as the given withdrawal transaction is registered.

2) The deposit transactions data are grouped by their address. Addresses that deposited the same number of times as the number of withdraws registered, are grouped in a set $C$.

3) An anonimity score (of this heuristic) is assigned to the withdrawal transaction following the formula $P = 1 - 1/|C|$, where P is the anonimity score and $|C|$ is the cardinality of set $C$.

In [1]:
# Set environment variables to better visualize DataFrames.
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

In [2]:
# Import relevant packages.
using DataFrames
using CSV
using StatsPlots
using ProgressBars

In [3]:
# Load withdraw and deposit data.
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame);

In [4]:
tornado_addresses = Dict(
    "0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3" => "100 DAI",
    "0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144" => "1000 DAI",
    "0x07687e702b410fa43f4cb4af7fa097918ffd2730" => "10000 DAI",
    "0x23773e65ed146a459791799d01336db287f25334" => "100000 DAI",
    "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc" => "0.1 ETH",
    "0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936" => "1 ETH",
    "0x910cbd523d972eb0a6f4cae4618ad62622b39dbf" => "10 ETH",
    "0xa160cdab225685da1d56aa342ad8841c3b53f291" => "100 ETH",
    "0xd96f2b1c14db8458374d9aca76e26c3d18364307" => "100 USDC",
    "0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d" => "1000 USDC",
    "0x169ad27a470d064dede56a2d3ff727986b15d52b" => "100 USDT",
    "0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f" => "1000 USDT",
    "0x178169b423a011fff22b9e3f3abea13414ddd0f1" => "0.1 WBTC",
    "0x610b717796ad172b316836ac95a2ffad065ceab4" => "1 WBTC",
    "0xbb93e510bbcd0b7beb5a853875f9ec60275cf498" => "10 WBTC",
    "0x22aaa7720ddd5388a3c0a3333430953c68f1849b" => "5000 cDAI",
    "0x03893a7c7463ae47d46bc7f091665f1893656003" => "50000 cDAI",
    "0x2717c5e28cf931547b621a5dddb772ab6a35b701" => "500000 cDAI",
    "0xd21be7248e0197ee08e0c20d4a96debdac3d20af" => "5000000 cDAI"
    )    

Dict{String, String} with 19 entries:
  "0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3" => "100 DAI"
  "0x07687e702b410fa43f4cb4af7fa097918ffd2730" => "10000 DAI"
  "0x22aaa7720ddd5388a3c0a3333430953c68f1849b" => "5000 cDAI"
  "0xd96f2b1c14db8458374d9aca76e26c3d18364307" => "100 USDC"
  "0xbb93e510bbcd0b7beb5a853875f9ec60275cf498" => "10 WBTC"
  ⋮                                            => ⋮

In [5]:
function compare_transactions(withdraws_dict, deposits_dict)
    for currency in keys(withdraws_dict)
        if !(deposits_dict[currency] >= withdraws_dict[currency])
            return false
        end
    end
    return true
end

compare_transactions (generic function with 1 method)

### Function summary: get_number_of_withdraws
Given a withdraw transaction and the total withdrawals data, the function get_number_of_withdraws returns the number of previous withdrawal transactions with the same address as the withdraw transaction.

In [6]:
function get_number_of_withdraws(withdraw_transaction, withdraw_transactions_df, tornado_addresses)
    
    # The number of withdraws is initialized at 1 since the withdraw_transaction of the first argument is always present
    # in the withdrawal data. Also, the count should be 1 if there is no other transaction with the same address.
    
    n_withdraws = Dict(tornado_addresses[withdraw_transaction.tornado_cash_address] => 1)
    
    # This for loop counts the number of transactions with the same address. At the end, the total number is returned.
    # The count is done considering that the recipient_address of each of the transactions in the withdraw_transactions_df
    # is the same as the recipient_address of the withdraw_transaction input, and that the timestamp of the rows is earlier
    # than the withdraw_transaction input. 
    # The if clause also filters by the transaction hash, since we don't want to count the same transaction two times.
    
    for row ∈ eachrow(withdraw_transactions_df)
        if (row.recipient_address == withdraw_transaction.recipient_address) && 
            (row.block_timestamp <= withdraw_transaction.block_timestamp) && 
            (row.hash != withdraw_transaction.hash)
            
            if haskey(n_withdraws, tornado_addresses[row.tornado_cash_address])
                n_withdraws[tornado_addresses[row.tornado_cash_address]] += 1
            else
                n_withdraws[tornado_addresses[row.tornado_cash_address]] = 1
            end
        end
    end
    return n_withdraws
end

get_number_of_withdraws (generic function with 1 method)

In [7]:
function get_address_deposits(deposit_transactions_df, tornado_addresses)
    # unique_addresses = unique(deposit_transactions_df[!, "from_address"])
    
    addresses_and_deposits_counts = combine(groupby(deposit_transactions_df, [:from_address, :tornado_cash_address]), nrow => :count)
    
    addresses_and_deposit_dict = Dict()
    for row in eachrow(addresses_and_deposits_counts)
        if haskey(addresses_and_deposit_dict, row.from_address)
             if haskey(addresses_and_deposit_dict[row.from_address], tornado_addresses[row.tornado_cash_address])
                 addresses_and_deposit_dict[row.from_address][tornado_addresses[row.tornado_cash_address]] += row.count
             else
                addresses_and_deposit_dict[row.from_address][tornado_addresses[row.tornado_cash_address]] = row.count
             end
        else
            addresses_and_deposit_dict[row.from_address] = Dict(tornado_addresses[row.tornado_cash_address] => row.count)
        end
    end
    
    return addresses_and_deposit_dict
end

get_address_deposits (generic function with 1 method)

In [8]:
# test
get_address_deposits(deposit_transactions_df, tornado_addresses)

Dict{Any, Any} with 23504 entries:
  "0x23f11f0571e392390929ec909be393fd93e79eb6" => Dict("1 ETH"=>2)
  "0x390e817d1c8f764dd4228d67dfbe85ee5de2fbcf" => Dict("10 ETH"=>1, "100 ETH"=>1)
  "0xa3a0a77d44ab389b1bbfdd6305a82a872449c8a1" => Dict("10 ETH"=>9, "1 ETH"=>8)
  "0x7b590b11f20f5e942f7b39f05b6827f4787cf435" => Dict("100000 DAI"=>1)
  "0x110790db185798abc9fc14570a60eab346ad6dbd" => Dict("1 ETH"=>1)
  ⋮                                            => ⋮

In [9]:
addresses_and_deposits_counts = combine(groupby(deposit_transactions_df, [:from_address, :to_address]), nrow => :count)

,from_address,to_address,count
,String,String,Int64
1,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,3
2,0x6e92bc493c6abbdd6a1b18416f003de2c873ab50,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,2
3,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,2
4,0x50b9d4af009b038506d4d84b035c451d1a3a20bc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,3
5,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,7
6,0x23480df691dbf7c62e967952bbf2067c18cc2f16,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,2
7,0x5ba446670288149052645705618f121af76dd19d,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,4
8,0xd0698d231d4b65b97a3df9c16aafda8d9b0bda41,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,1
9,0x3a456bc9083bfe147719504aee8f296eb7355ee1,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,1


### Function summary: get_same_number_of_deposits
Given a number of withdrawal transactions, the function registers all the addresses that have made that same number of deposits. Returns an array with all the addresses that match these requirements.

In [10]:
function get_same_number_of_deposits(n_withdraws, deposit_transactions_df, tornado_addresses)
    
    # The deposits transactions data is first grouped by address, and then combined to get a new 
    # DataFrame with the addresses and their corresponding counts.
    
    address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)
    
    return filter(address_deposit -> (Set(keys(n_withdraws))) == Set(keys(last(address_deposit))) &&
    compare_transactions(n_withdraws, last(address_deposit)),
    address_deposits) |> keys |> collect
    
    
    # The addresses are filtered by the number of counts. Only the ones that are equal or larger than the input
    # n_withdraws are returned. Finally, the output of the function is an array with the matching addresses.
    
    # return filter(row -> row.count >= n_withdraws[tornado_addresses[row.to_address]], addresses_and_deposit_counts)[!, :from_address]
end

get_same_number_of_deposits (generic function with 1 method)

In [11]:
xx = get_same_number_of_deposits(c, deposit_transactions_df, tornado_addresses)

LoadError: UndefVarError: c not defined

In [12]:
address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)

Dict{Any, Any} with 23504 entries:
  "0x23f11f0571e392390929ec909be393fd93e79eb6" => Dict("1 ETH"=>2)
  "0x390e817d1c8f764dd4228d67dfbe85ee5de2fbcf" => Dict("10 ETH"=>1, "100 ETH"=>1)
  "0xa3a0a77d44ab389b1bbfdd6305a82a872449c8a1" => Dict("10 ETH"=>9, "1 ETH"=>8)
  "0x7b590b11f20f5e942f7b39f05b6827f4787cf435" => Dict("100000 DAI"=>1)
  "0x110790db185798abc9fc14570a60eab346ad6dbd" => Dict("1 ETH"=>1)
  ⋮                                            => ⋮

In [25]:
# test
a = Dict("0.1 ETH" => 4)
c = Dict("10 ETH" => 1, "100 ETH" => 1, "1 ETH" => 1, "0.1 ETH" => 1)
address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)

filter(t_count_dict -> (Set(keys(c))) == Set(keys(last(t_count_dict))) &&
    compare_transactions(c, last(t_count_dict)),
    address_deposits)
    

Dict{Any, Any} with 174 entries:
  "0x40c49abe80ba30d9bbfffbd69e56e703278b3dcf" => Dict("10 ETH"=>4, "100 ETH"=>7, "1 ETH"=>9, "0.1 ETH"=>4)
  "0x4b285130a1afaac88b46e6e643b1e123b7b36ada" => Dict("10 ETH"=>9, "1 ETH"=>5, "100 ETH"=>2, "0.1 ETH"=>1)
  "0x03079442ec8ad4fa53458336087c405525e9a75b" => Dict("10 ETH"=>6, "100 ETH"=>5, "1 ETH"=>5, "0.1 ETH"=>2)
  "0x5ebc7d1ff1687a75f76c3edfabcde89d1c09cd5f" => Dict("10 ETH"=>5, "100 ETH"=>1, "1 ETH"=>3, "0.1 ETH"=>4)
  "0x8efdc05de2071ea4ca22e73fc5ab5e4f57988ee7" => Dict("10 ETH"=>1, "100 ETH"=>4, "1 ETH"=>1, "0.1 ETH"=>4)
  ⋮                                            => ⋮

In [14]:
a = collect(keys(address_deposits))[803]
address_deposits[a]
c = Dict("10 ETH" => 1, "100 ETH" => 1, "1 ETH" => 1, "0.1 ETH" => 1)

Dict{String, Int64} with 4 entries:
  "10 ETH"  => 1
  "100 ETH" => 1
  "1 ETH"   => 1
  "0.1 ETH" => 1

### Function summary: get_same_number_of_deposits_heuristic
Given a withdraw_transaction and the deposit and withdraw data, compute the anonimity score of the transaction, based on this heuristic.

In [15]:
function same_number_of_deposits_heuristic(withdraw_transaction, deposit_transactions_df, withdraw_transactions_df, tornado_addresses)
                                            
    # We calculate the number of withdrawals of the address from the withdraw_transaction given as input.
    
    n_withdraws = get_number_of_withdraws(withdraw_transaction, withdraw_transactions_df, tornado_addresses)
        
    # Based on n_withdraws, the set of the addresses that have the same number of deposits is calculated.
    
    Ϛ = get_same_number_of_deposits(n_withdraws, deposit_transactions_df, tornado_addresses)
    
    # The anonimity score P is computed.
    
    P = 1 - 1/length(Ϛ)
    
    # Since there is a chance that the cardinality of the set is 0, we handle this case and return the anonimity
    # score.
    
    return isinf(P) ? 1 : P
end

same_number_of_deposits_heuristic (generic function with 1 method)

In [16]:
# Just for testing an example, we select a random withdraw_transaction
withdraw_transaction = withdraw_transactions_df[11,:]

,Column1,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
,Int64,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Float64?,Float64?,Float64?,Int64,String,String
11,10,45,0x795de0514ec2b5e9421a0730c5d837c4ac3387d7cfb19f9f81f00601d9a781ac,258,57,0xefab18983029d2ba840e34698efb67fdf8120711,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,383057,5050000000,4097750,333057,missing,missing,1,2020-02-04 04:15:55 UTC,9413844,0x0e9f185b6277a831eb1bc6419df0b8060daebb81a46912f6374d0bff41e1b59f,missing,missing,missing,5050000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0xeab368d778c604a7dab378f736c8c3f64123073a


In [17]:
get_number_of_withdraws(withdraw_transaction, withdraw_transactions_df, tornado_addresses)

Dict{String, Int64} with 1 entry:
  "10 ETH" => 1

Testing the function get_same_number_of_deposits. We can see there are 3 addresses that have done 34 deposits.


In [18]:
# test
deposits_with_same_number = combine(groupby(deposit_transactions_df, [:from_address, :to_address]), nrow => :count)
unique(deposit_transactions_df[!, "from_address"])

23504-element Vector{String}:
 "0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5"
 "0x6e92bc493c6abbdd6a1b18416f003de2c873ab50"
 "0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8"
 ⋮
 "0xd29dcbd273d73714eb6bea7bd4f8d99d0fbc3398"
 "0xb12814fdbbdc8c8f9c2741bde788813a90588be6"

Testing same_number_of_deposits_heuristic with the withdraw_transaction we selected, computes an anonimity score to it.

In [24]:
same_number_of_deposits_heuristic(withdraw_transaction, deposit_transactions_df, withdraw_transactions_df, tornado_addresses)

0.9997814685314685

In [20]:
function apply_same_number_of_deposits_heuristic(deposit_transactions_df, withdraw_transactions_df, tornado_addresses)
    
    address_deposits = get_address_deposits(deposit_transactions_df, tornado_addresses)
    
    # An empty dictionary is initialized. It will be used to store the anonimity score associated with each transaction.
    
    tx_hash_and_anonimity_score = Dict()
    
    # Iterate over every row of the withdraw_transactions DataFrame and apply the function same_number_of_deposits_heuristic
    # For each transaction, the anonimity score is computed and appended to the dictionary.
    
    for withdraw_row ∈ ProgressBar(eachrow(withdraw_transactions_df), printing_delay=5)
        anonimity_score = same_number_of_deposits_heuristic(withdraw_row, deposit_transactions_df, withdraw_transactions_df, tornado_addresses)
        tx_hash_and_anonimity_score[withdraw_row.hash] = anonimity_score
    end
    
    # The dictionary with the transaction hashes and scores is returned
    
    tx_hash_and_anonimity_score
end

apply_same_number_of_deposits_heuristic (generic function with 1 method)

We apply this last function to our data and we obtain a dictionary indicating the score of each withdrawal.

In [21]:
scores = apply_same_number_of_deposits_heuristic(deposit_transactions_df, withdraw_transactions_df, tornado_addresses)


0.0%┣                                     ┫ 0/83.8k [00:05<-116:-21:-50, -5s/it]
0.0%┣                                        ┫ 1/83.8k [00:05<Inf:Inf, InfGs/it]
0.1%┣                                         ┫ 45/83.8k [00:10<05:32:09, 4it/s]
0.1%┣                                         ┫ 90/83.8k [00:16<04:03:35, 6it/s]
0.2%┣                                        ┫ 134/83.8k [00:21<03:35:32, 6it/s]
0.2%┣                                        ┫ 178/83.8k [00:26<03:21:17, 7it/s]


LoadError: InterruptException:

In [22]:
score_values = values(scores) |> collect
histogram(score_values, bins=500, legend=false, title="Histogram of scores of the withdrawal transactions", xlabel="Scores")

LoadError: UndefVarError: scores not defined